In [1]:
import json
import sqlite3
from arcgis.gis import GIS
from arcgis.geometry import Point, Polyline
from arcgis.features import Feature, FeatureSet
from komiwojazer import Komiwojazer
from graf import Graf
from wyszukiwanie_pliku import sciezka_do_katalogu_gl_projektu
import pandas as pd

def pobieranie_danych_od_uzytkownika():
    """
    Metoda służąca do pobierania danych od użytkownika dot. miasta skąd chcą zacząc wycieczkę i jakie miasta odwiedzić
    :return miasto_startowe: zwraca miasto startowe podróży
    :return miasta_do_odwiedzenia: zwraca miasta do odwiedzenia podczas podróży
    """
    miasto_startowe = int(input("Podaj miasto początkowe podróży po Europie (1-20): "))  # pobieranie danych od użytkownika
    miasta_do_odwiedzenia = input("Podaj jakie miasta europejskie chcesz odwiedzić (1-20): ")
    miasta_do_odwiedzenia = list(map(int, miasta_do_odwiedzenia.split(',')))  # int - tylko liczby całkowite, dzielenie łańcucha znaków na podłańcuchy używając przecinka jako separator, np.: lista: 1,2,3 -> '1','2','3'
    return miasto_startowe, miasta_do_odwiedzenia

komiwojazer = Komiwojazer()
graf = Graf()

projekt = sciezka_do_katalogu_gl_projektu()
plik = open(projekt/ "macierz_sasiedztwa_grafu.json")
macierz_sasiedztwa_grafu = json.load(plik)


miasto_startowe, miasta_do_odwiedzenia = pobieranie_danych_od_uzytkownika()

# podgraf
_, podgraf = graf.tworzenie_grafu(macierz_sasiedztwa_grafu, miasto_startowe, miasta_do_odwiedzenia) # _ używane do oznaczania zmiennych, które nie będą używane
macierz, oryginalne_wierzcholki, oryginalne_wierzcholki_konwersja = graf.konwersja(podgraf)

# graf
graf_pelny, _ = graf.tworzenie_grafu(macierz_sasiedztwa_grafu, miasto_startowe, [i for i in range(1, miasto_startowe)] + [i for i in range(miasto_startowe + 1, 21)])
macierz_pelna, org_wierzch_peln, org_wierzch_konw_peln = graf.konwersja(graf_pelny)


# Kruksal
trasa_kr, dystans_kr = komiwojazer.tsp(macierz, oryginalne_wierzcholki[miasto_startowe])
for i in range(len(trasa_kr)):
    trasa_kr[i] = oryginalne_wierzcholki_konwersja[trasa_kr[i]]

# Najbliższy sąsiad
trasa_nb, dystans_nb = komiwojazer.najblizszy_sasiad(macierz, oryginalne_wierzcholki[miasto_startowe])
for i in range(len(trasa_nb)):
    trasa_nb[i] = oryginalne_wierzcholki_konwersja[trasa_nb[i]]

# Held-Karp
trasa_hk, dystans_hk, opt = komiwojazer.held_karp(macierz, oryginalne_wierzcholki[miasto_startowe])
for i in range(len(trasa_hk)):
    trasa_hk[i] = oryginalne_wierzcholki_konwersja[trasa_hk[i]]

# Naiwny
trasa_naiw, dystans_naiw = komiwojazer.naiwny(macierz, oryginalne_wierzcholki[miasto_startowe])
for i in range(len(trasa_naiw)):
    trasa_naiw[i] = oryginalne_wierzcholki_konwersja[trasa_naiw[i]]

# tworzenie tabeli
wyniki = {
"Algorytm": ["Kruksal", "Najbliższy Sąsiad", "Held-Karp", "Naiwny"],
"Trasa": [trasa_kr, trasa_nb, trasa_hk, trasa_naiw],
# "Dystans [km]": [dystans_kr, dystans_nb, dystans_hk, dystans_naiw]
}

df = pd.DataFrame(wyniki)

df.index = range(1, len(df) + 1) # ustawienie indeksowania od 1

# ustawienie stylu tabeli
styl_df = df.style.set_properties(**{
    'border': '1px solid pink',
    'color': '#FF1493',
    'font-size': '14px'
}).format({'Dystans [km]': '{:.2f}'})  # dystans do dwóch miejsc po przecinku

# Wyświetlenie tabeli
print("\n")
print("Tabela wyników:")
display(styl_df)

print("\n")
rysowanie_podgrafu = graf.rysowanie_grafu(macierz, oryginalne_wierzcholki[miasto_startowe], oryginalne_wierzcholki_konwersja)

print("\n")
rysowanie_grafu = graf.rysowanie_grafu(macierz_pelna, org_wierzch_peln[miasto_startowe], org_wierzch_konw_peln)

# GIS

# Połączenie z bazą danych
polaczenie = sqlite3.connect('baza.db')
kursor = polaczenie.cursor()
kursor.execute("SELECT ID, y, x, miasto FROM lotniska")
miasta = kursor.fetchall()

# klucz arcgis api
gis = GIS(api_key="AAPKa395ddfad8ab4d8da56e18f2da515c340n0QtN2aJgsETPaNHBzNxCcV3mMr5YJoh5m9lkdrYiDtiVqOmMIkSz6z8AW0_CQQ")

def tworzenie_warstwy_trasa(trasy: list, nazwy_alg: list) -> list:
    '''
    Metoda służąca do tworzenia warstw zawierających trasy w formie polilinii wyznaczone przez algorytmy komiwojażera
    :param trasy: lista tras wyznaczonych przez algorytmy komiwojażera
    :param nazwy_alg: lista nazw algorytmów rozwiązujących problem komiwojażera
    :return: lista polilinii do narysowania na mapie
    '''

    warstwy = []
    kolory = [
    [242, 219, 124],
    [190, 242, 155],
    [155, 206, 242],
    [242, 155, 203]
    ]

    # Iteracja przez trasy i konkretny algorytm
    for trasa, nazwa_alg, kolor in zip(trasy, nazwy_alg, kolory):
        # Tworzenie listy współrzędnych miast
        miasta_polaczenia = [(miasta[i - 1][2], miasta[i - 1][1]) for i in trasa]

        # Tworzenie polilinii na podstawie listy współrzędnych
        polaczenie_trasa = Polyline({
            "paths": [miasta_polaczenia],
            "spatialReference": {"wkid": 4326}
        })

        # Tworzenie opisu trasy
        opis_trasy = " ✈ ".join([f"{id + 1}. {miasta[i - 1][3]}" for id, i in enumerate(trasa)])

        # Aktualizacja atrybutów trasy
        trasa_attributes = {"Nazwa algorytmu": nazwa_alg, "Szczegóły trasy": opis_trasy}

        # Tworzenie warstwy dla trasy
        warstwa_trasa = Feature(geometry=polaczenie_trasa, attributes=trasa_attributes)
        warstwy.append(warstwa_trasa)

    return FeatureSet(warstwy)

def tworzenie_warstwy_miasta(trasy: list, nazwy_alg: list) -> FeatureSet:
    '''
    Metoda służąca do tworzenia warstw zawierających punkty, które reprezentują miasta w trasie
    :param trasy: lista tras wyznaczonych przez algorytmy komiwojażera
    :param nazwy_alg: lista nazw algorytmów rozwiązujących problem komiwojażera
    :return: zwracana jest warstwa zawierająca miasta trasy w formie punktów
    '''
    warstwy = []

    # Iteracja przez trasy
    for trasa, nazwa_alg in zip(trasy, nazwy_alg):

        # Iteracja przez miasta w trasie
        for id, i in enumerate(trasa):
            # Tworzenie punktu dla miasta
            miasta_trasy = Point({"x": miasta[i - 1][2], "y": miasta[i - 1][1], "spatialReference": {"wkid": 4326}})

            # Tworzenie atrybutów dla miasta
            miasto_attributes = {"Label": f"{nazwa_alg} - {id + 1}", "Miasto": miasta[i - 1][3]}

            # Tworzenie warstwy dla miasta
            warstwa_miasta = Feature(geometry=miasta_trasy, attributes=miasto_attributes)
            warstwy.append(warstwa_miasta)

    return FeatureSet(warstwy)

trasy = [trasa_nb, trasa_hk, trasa_naiw, trasa_kr]
nazwy_alg = ["Najbliższy sąsiad", "Held-Karp", "Naiwny", "Kruskal"]
warstwy_trasy = tworzenie_warstwy_trasa(trasy, nazwy_alg)
warstwa_miasta = tworzenie_warstwy_miasta(trasy, nazwy_alg)

mapa = gis.map()
mapa.basemap = "topo-vector"

kolory = [
    [242, 219, 124],
    [190, 242, 155],
    [155, 206, 242],
    [242, 155, 203]
]

for id, warstwa in enumerate(warstwy_trasy.features):
    title = warstwa.attributes.get('Nazwa algorytmu', 'Brak tytułu')
    content = warstwa.attributes.get('Szczegóły trasy', 'Brak zawartości')

    kolor = kolory[id]

    mapa.draw(
        shape=warstwa.geometry,
        symbol={
            "type": "esriSLS",
            "style": "esriSLSolid",
            "color": kolor,  # Wybrany kolor
            "width": 2
        },
        attributes=warstwa.attributes,
        popup={"title": title, "content": content}
    )

# Dodanie warstwy miast do mapy
mapa.add_layer(warstwa_miasta)
mapa.popup = {"title": "{Label}", "content": "Miasto: {Miasto}"}

display(mapa)

KeyboardInterrupt: Interrupted by user